# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [10]:
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core import ScriptRunConfig
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.automl import AutoMLConfig
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
from azureml.data.dataset_factory import TabularDatasetFactory
from sklearn.model_selection import train_test_split
from trainCovid19Dataset import clean_data
import os
import shutil

## Initialize Workspace

Create a workspace, if it doesn't exist, using the AzureML SDK

In [11]:
ws = Workspace.from_config()
    
ws.get_details()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

Workspace name: quick-starts-ws-140100
Azure region: southcentralus
Subscription id: 81cefad3-d2c9-4f77-a466-99a7f541c7bb
Resource group: aml-quickstarts-140100


## Cluster

Get cluster if it exists else create one

In [12]:
amlcompute_cluster_name = "Covid19Cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Dataset

In [19]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
found = False
key = "Covid19Dataset"
description_text = "Covid19 Vaccination DataSet from Kaggle"
datastore = ws.get_default_datastore()
datastore.upload_files(files = ['./starter_file/kaggle/country_vaccinations.csv'],
                       target_path ='train-dataset/tabular/',
                       overwrite = True,
                       show_progress = True)
if key in ws.datasets.keys(): 
    found = True
    dataset = ws.datasets[key] 

if not found:
    os.environ['KAGGLE_USERNAME']= 'ocherif'
    os.environ['KAGGLE_KEY']= 'cb037f99cae382b7a67c68f8048e01be'
    import kaggle
    kaggle.api.authenticate()
    kaggle.api.dataset_download_files('gpreda/covid-world-vaccination-progress', path='./starter_file/kaggle', unzip=True)
    
    ds = Dataset.Tabular.from_delimited_files(path = [(datastore, 'train-dataset/tabular/country_vaccinations.csv')])
    
    #Register Dataset in Workspace
    dataset = ds.register(workspace=ws,
                          name=key,
                          description=description_text)

df = dataset.to_pandas_dataframe().fillna(0)
df.describe()

Uploading an estimated of 1 files
Uploading ./starter_file/kaggle/country_vaccinations.csv
Uploaded ./starter_file/kaggle/country_vaccinations.csv, 1 files out of an estimated total of 1
Uploaded 1 files


,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million
count,5.457000e+03,5.457000e+03,5.457000e+03,5.457000e+03,5.457000e+03,5457.000000,5457.000000,5457.000000,5457.000000
mean,1.268187e+06,9.322980e+05,2.314462e+05,4.309916e+04,5.563308e+04,5.081378,3.559987,1.013826,2451.715778
std,5.533060e+06,4.001554e+06,1.520050e+06,1.750228e+05,1.816005e+05,12.498955,8.415156,4.040757,4366.573612
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000
25%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,9.640000e+02,0.000000,0.000000,0.000000,291.000000
50%,2.342900e+04,8.669000e+03,0.000000e+00,1.770000e+02,5.508000e+03,0.430000,0.120000,0.000000,1057.000000
75%,4.387080e+05,2.761030e+05,2.482700e+04,1.506700e+04,2.648600e+04,4.560000,3.150000,0.540000,2509.000000
max,9.035175e+07,5.887371e+07,3.068688e+07,2.904229e+06,2.159392e+06,129.740000,83.520000,46.220000,54264.000000


In [20]:
ws = Workspace.from_config()
experiment_name = 'Covid19HyperDrExperiment'

experiment=Experiment(ws, experiment_name)

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [21]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(slack_factor = 0.2, evaluation_interval=2, delay_evaluation=5)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling( 
    {
        '--C': choice(0.01, 0.1, 1, 10, 100), 
        '--max_iter': choice(25, 50, 100,150)
    }
)
if "training" not in os.listdir():
    os.mkdir("./training")
script_folder = './training/'    
os.makedirs(script_folder, exist_ok=True)
shutil.copy('trainCovid19Dataset.py', script_folder)

#TODO: Create your estimator and hyperdrive config
estimator = SKLearn(source_directory=script_folder, compute_target= amlcompute_cluster_name, entry_script='trainCovid19Dataset.py')

hyperdrive_run_config = HyperDriveConfig(
                                   hyperparameter_sampling = param_sampling,
                                   primary_metric_name = 'Accuracy',
                                   primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                   max_total_runs=30,
                                   max_concurrent_runs = 3,
                                   policy = early_termination_policy,
                                   estimator = estimator)

In [22]:
#TODO: Submit your experiment
tag = {"Covid19": "Capstone project: Covid19 HyperDrive Experiment"}
remote_run = experiment.submit(hyperdrive_run_config,tags=tag, show_output=True)


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [23]:
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_173b6fc0-eb45-4806-b84c-18c09b350847
Web View: https://ml.azure.com/experiments/Covid19HyperDrExperiment/runs/HD_173b6fc0-eb45-4806-b84c-18c09b350847?wsid=/subscriptions/81cefad3-d2c9-4f77-a466-99a7f541c7bb/resourcegroups/aml-quickstarts-140100/workspaces/quick-starts-ws-140100

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-03-09T16:26:13.938516][API][INFO]Experiment created<END>\n""<START>[2021-03-09T16:26:14.573409][GENERATOR][INFO]Trying to sample '3' jobs from the hyperparameter space<END>\n""<START>[2021-03-09T16:26:14.719314][GENERATOR][INFO]Successfully sampled '3' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2021-03-09T16:26:15.1292366Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>


In [ ]:
import joblib
# Get your best run and save the model from that run.
best_run_HyperDrive = hyperdrive_run.get_best_run_by_primary_metric()

SKLearn_Model = best_run_HyperDrive.register_model(model_name="HyperCovid19VaccinationModel", model_path='outputs/model.joblib')
# Get the metrics of the bestselected run
best_run_metrics = best_run_HyperDrive.get_metrics()
# Show the Accuracy of that run
print('Best Accuracy: {}'.format(best_run_metrics['Accuracy']))
best_run_HyperDrive

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
#TODO: Save the best model

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service